In [1]:
import numpy as np
import pandas as pd
import paho.mqtt.client
import radiotracking
import datetime

import io
import collections

import plotly.graph_objs as go

In [2]:
# init data 
data = collections.namedtuple("data", ["df", "fig", "last_plot"])
data.df = pd.DataFrame(index=[], columns=["SDR", *radiotracking.Signal.header])
data.last_plot = datetime.datetime.now()

In [3]:
data.fig = go.FigureWidget()

data.fig.add_trace(go.Scatter(name="SDR 0", mode='markers'))
data.fig.add_trace(go.Scatter(name="SDR 1", mode='markers'))
data.fig.add_trace(go.Scatter(name="SDR 2", mode='markers'))
data.fig.add_trace(go.Scatter(name="SDR 3", mode='markers'))

data.fig.data[0].marker.color="green"
data.fig.data[1].marker.color="red"
data.fig.data[2].marker.color="orange"
data.fig.data[3].marker.color="blue"

In [4]:
def on_message(client, data, message):
    msg_df = pd.read_csv(io.StringIO(message.payload.decode()), names=radiotracking.Signal.header, parse_dates=["Time"])
    sdr = message.topic.split("/")[-1]
    msg_df["SDR"] = sdr
    data.df = data.df.append(msg_df, ignore_index=True, sort=True)
    
    now = msg_df["Time"].iloc[-1]
    
    if data.last_plot < now - datetime.timedelta(seconds=2):
        data.last_plot = now
        #gdf = data.df.groupby("SDR").get_group(sdr)
        for sdr, gdf in data.df.groupby("SDR"):
            data.fig.data[int(sdr)].x = gdf["Time"].iloc[-20:]
            data.fig.data[int(sdr)].y = gdf["avg (dBW)"].iloc[-20:]

    return msg_df

In [5]:
client = paho.mqtt.client.Client(userdata=data)
client.connect("localhost")
client.on_message = on_message
client.subscribe("nature40-sensorbox-ec8331f4/radiotracking/csv/#", qos=0)

display(data.fig)
client.loop_forever(timeout=1.0, max_packets=1, retry_first_connection=False)

FigureWidget({
    'data': [{'marker': {'color': 'green'},
              'mode': 'markers',
              'nam…

KeyboardInterrupt: 